In [48]:
# adds parent dir to python path
import sys
sys.path.insert(0, '..')

#system lib
import os
import json

# 3rd party lib
import numpy as np
#import matplotlib
#import matplotlib.pyplot as plt
#%matplotlib notebook
#matplotlib.rcParams.update({'font.size': 22})
#from bqplot import pyplot as plt

import matplotlib.pyplot as plt
%matplotlib auto

# astro lib
from astropy.io import fits
import sep

# my lib
import ImageTools as it
import DataTools as dt
from gphelper import GPHelper

from importlib import reload

import warnings
warnings.filterwarnings('ignore')

Using matplotlib backend: Qt5Agg


In [47]:
def make_nonnegative(img):
    epsilon = np.abs(img.min()) + 1e-3
    img[img<=0] = img[img>0].min()
    return img

def get_random_spheroids(num=10):
    with open('../spheroids', 'r') as f:
        spheroids = np.array(f.readlines())
    
    if num==-1:
        num = len(spheroids)
    
    selected = np.random.choice(spheroids, num, replace=False)
    
    data_dir = os.path.join(os.getenv('HOME'), 'Documents/astro_data/orig_images')
    fmask = 'GDS_{}_{}.fits'
    f_string = os.path.join(data_dir, fmask)

    sources = []
    for s in selected:
        img = fits.getdata(f_string.format(s.strip(), 'h'))
        segmap = fits.getdata(f_string.format(s.strip(), 'segmap'))
        img_id = int(s.split('_')[1])
        sources.append((img.copy(), segmap.copy(), img_id))
        del img 
        del segmap
    
    return sources

# Petrosian Helpers ----------------------------------------------------------
def one_over_eta(rs, fs, R):
    return ((np.pi*R**2) / (np.sum(fs[rs<=R]))) * fs[rs==R]

def petrosian_radius(rs, fs):
    #r_candidates = np.array([one_over_eta(rs, fs, R) for R in rs])
    r_candidates = np.array([fs[rs==r]/fs[rs<=r].mean() for r in rs]).flatten()
    r_candidates = r_candidates[rs<12.5]
    
    return rs[np.square(r_candidates-0.2).argmin()]

def petrosian_flux(rs, fs, R_p):
    return np.sum(fs[rs<=2*R_p])

def R_x(rs, fs, x):
    x /= 100
    sum_ratio = np.cumsum(fs)/np.sum(fs)
    return rs[np.square(sum_ratio-x).argmin()]

def petrosian_Re(R_50, R_90):
    P_3 = 8e-6
    P_4 = 8.47
    R_ratio = min(R_90/R_50, 3.8)
    return R_50 / (1 - (P_3 * (R_ratio)**P_4))

def get_rs_and_fs(img, src_map):
    cx, cy = it.img_center(img, src_map)
    xs, ys = np.meshgrid(np.arange(img.shape[0]), np.arange(img.shape[1]).T)
    rs = np.sqrt(np.square(xs-cx)+np.square(ys-cy))
    
    rs = rs.flatten()
    fs = img.flatten()
    sorted_rs = np.argsort(rs)
    
    rs = rs[sorted_rs]
    fs = fs[sorted_rs]
    
    return rs, fs

def get_re(img, src_map):
    rs, fs = get_rs_and_fs(img, src_map)
    
    R_p = petrosian_radius(rs, fs)
    F_p = petrosian_flux(rs, fs, R_p)
    p_mask = rs <= 2*R_p
    
    _rs = rs[p_mask]
    _fs = fs[p_mask]

    R_50 = R_x(_rs, _fs, 50)
    R_90 = R_x(_rs, _fs, 90)

    re = petrosian_Re(R_50, R_90)
    re_idx = np.square(_rs-re).argmin()
    
    return rs[re_idx], rs, fs

def get_re(rsfs):
    rs, fs = rsfs
    
    R_p = petrosian_radius(rs, fs)
    F_p = petrosian_flux(rs, fs, R_p)
    p_mask = rs <= 2*R_p
    
    _rs = rs[p_mask]
    _fs = fs[p_mask]

    R_50 = R_x(_rs, _fs, 50)
    R_90 = R_x(_rs, _fs, 90)

    re = petrosian_Re(R_50, R_90)
    re_idx = np.square(_rs-re).argmin()
    
    return rs[re_idx]

# Petrosian Helpers ----------------------------------------------------------

def denoise(img, segmap, img_id):
    noise_bank = img[segmap==0].flatten()
    other_source = np.logical_and(segmap!=0, segmap!=img_id)
    np.place(img, other_source, noise_bank)
    
    array_sep_likes = img.byteswap().newbyteorder()
    bkg = sep.Background(array_sep_likes, mask=segmap==img_id, bw=10, bh=10)
    img = img-bkg
    
    return img

# https://ned.ipac.caltech.edu/level5/March05/Graham/Graham2.html
def sersic(x):
    def b(n):
        return 1.9992*n-0.3271

    def I(r):
        """Assuming that I_e=1.0 and that R_e=1.0"""
        n = 4
        return np.exp(-b(n) * (np.power(r, 1/n)  - 1))

    return I(x)

In [49]:
reload(dt)

re_max_ratio = []
_spheroids = get_random_spheroids(num=-1)

res = []
ies = []
normed_rs = []
normed_fs = []
count = 0

small_res = []

for img, segmap, img_id in _spheroids:
    count += 1
    
    print(count/len(_spheroids), end='\r')

    #background subtract images
    src_map = segmap==img_id
    
    img = denoise(img, segmap, img_id)
    
    # get the sorted r vals and i vals
    rs, fs = get_rs_and_fs(img, src_map)
    
    # smooth the fs using loess
    #fs = dt.loessc(rs, fs, .60)
    fs = dt.loessc_p(rs, fs, 6.0, pnum=4)
    #print(len(rs), len(fs))
    #fs = lowess(fs, rs)
    #print(len(rs), len(fs))
    
    # measure petrosian re 
    re = get_re((rs,fs))

    
    if re<3:
        small_res.append((img_id, re))
        
    
    re_idx = np.where(rs==re)[0][0] 
    
    ie = fs[re_idx]
    
    #sum_to_re = fs[rs<=re_idx].sum()
    #src_L = sum_to_re*2
    #img_L = fs.sum()
    #if src_L>img_L:
    #    re_max_ratio.append(sum_to_re/img_L)
    #    continue
    
    #max_r = np.square(np.cumsum(fs)-src_L).argmin()
    
    #max_r = rs[max_r]
    
    #fs = fs
    #rs = rs
        
    normed_rs.append(rs)
    normed_fs.append(fs)
    ies.append(ie)
    res.append(re)    

In [ ]:
plt.title('Effective Radius')
plt.xlabel('$R_e$(Pixels)')
plt.hist(res, bins=100)


In [50]:
interped_marks = np.concatenate([np.linspace(0, 1, 50, endpoint=False), np.linspace(1, 20, 950)])

stat_vals = []

plt.figure()
plt.title('Normalized Surface Brightness')
plt.xlabel('$R/R_e$')
plt.ylabel('$I/I_e$')

for rs, fs, re, ie in zip(normed_rs, normed_fs, res, ies):
    color = 'b'
    if np.sum(fs<=0) > 0:
        fs = fs + np.abs(fs.min()) + 1e-3
        ie = fs[rs==re][0]
    
    r = rs/re
    f = fs/ie
    plt.semilogy(r, f, color='k', alpha=.05)
    stat_vals.append(np.interp(interped_marks, r, f, left=np.nan, right=np.nan))
    

stat_vals = np.dstack(stat_vals)[0,...]
f_mean = np.nanmedian(stat_vals, axis=1)
f_16 = np.nanpercentile(stat_vals, 16, axis=1)
f_84 = np.nanpercentile(stat_vals, 84, axis=1)
    
plt.semilogy(interped_marks, f_mean, color='r', label='$median$', zorder=100)
plt.fill_between(interped_marks, f_16, f_84, color='r', alpha=0.45, label="$16^{th}-84^{th}$", zorder=100)
plt.legend()
plt.show()

In [52]:
plt.scatter(res, ies)
plt.xlabel('$R_e$')
plt.ylabel('$I_e$')
plt.title('$R_e$ $I_e$ Correlation')
np.corrcoef(res, ies)

array([[ 1.        ,  0.35850239],
       [ 0.35850239,  1.        ]])

In [51]:
valid_points = ~np.isnan(stat_vals).all(axis=1)

X = interped_marks[valid_points, np.newaxis]
Y = f_mean[valid_points]
a = f_84[valid_points] - Y

vals = {'x':dt._nmpy_encode(X),
        'y':dt._nmpy_encode(Y),
        'a':dt._nmpy_encode(a)}
with open('vals_for_gp.json', 'w') as f:
    json.dump(vals,f)

In [37]:
with open('vals_for_gp.json', 'r') as f:
    vals = json.load(f)
    
X = dt._nmpy_decode(vals['x'])
Y = dt._nmpy_decode(vals['y'])
a = dt._nmpy_decode(vals['a'])

np.abs(np.square(X-1.0)).argmin()

989

In [56]:
_X = X[X<=5.0]
_Y = Y[X.flatten()<=5.0]
_a = a[X.flatten()<=5.0]
tmp = _Y-_a
plt.semilogy(_X,_Y)
tmp[tmp<=0]

plt.fill_between(_X, _Y-_a, _Y+_a, alpha=0.2)

In [44]:

gp = GPHelper()
gp.fit(X, Y, a, length_scale=0.75, optimize='sigma_n')
_x, std = gp.predict(X, return_std=True)
plt.semilogy(X,)
plt.semilogy(X,Y)

KeyboardInterrupt: 